In [ ]:
!pip install opencv-python mediapipe ultralytics

In [ ]:
!pip uninstall -y mediapipe
!pip install mediapipe==0.10.9 ultralytics opencv-python numpy

In [ ]:
import cv2
import os
import numpy as np
import mediapipe as mp
from ultralytics import YOLO
from google.colab import drive
drive.mount('/content/drive')

In [4]:
yolo_model_path = "/content/drive/MyDrive/yolov8n.pt"
if not os.path.exists(yolo_model_path):
    raise FileNotFoundError(f"YOLO model not found at {yolo_model_path}. Please check the path.")
model = YOLO(yolo_model_path)

In [5]:
# Initialize MediaPipe models
mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)
mp_holistic = mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_pose = mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_hands = mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [6]:
def detect_kidnapping(frame):
    """Detects signs of kidnapping using MediaPipe Pose & Hands."""

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pose_results = mp_pose.process(rgb_frame)
    hand_results = mp_hands.process(rgb_frame)

    risk_score = 0
    annotations = []

    # 1️⃣ **Detect Aggressive Hand Movements (Struggle)**
    if hand_results.multi_hand_landmarks:
        annotations.append(("Hand Detected - Possible Struggle", (10, 40), (0, 255, 255)))
        risk_score += 2  # Increase risk score

    # 2️⃣ **Detect Unusual Body Poses**
    if pose_results.pose_landmarks:
        left_wrist = pose_results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_WRIST]
        right_wrist = pose_results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.RIGHT_WRIST]
        left_shoulder = pose_results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = pose_results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER]

        # Check if hands are raised or crossed (defensive posture)
        if left_wrist.y < left_shoulder.y or right_wrist.y < right_shoulder.y:
            annotations.append(("Hands Raised - Potential Defense", (10, 70), (255, 0, 0)))
            risk_score += 3  # Higher risk

    # Add annotations to frame
    for text, pos, color in annotations:
        cv2.putText(frame, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    return frame, risk_score


In [7]:
def detect_kidnapping_in_video(video_path, output_video_path):
    """Processes video with YOLO + MediaPipe and saves output with risk analysis."""

    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video file: {video_path}")

    # Get video properties
    #fps - frames per second
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define Video Writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frame_count = 0
    skip_frames = 3  # Skip every 3 frames to improve efficiency

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames to speed up processing
        if frame_count % skip_frames == 0:
            # 1️⃣ Detect objects with YOLO
            yolo_results = model(frame)
            annotated_frame = yolo_results[0].plot()

            # 2️⃣ Detect kidnapping risk with MediaPipe
            annotated_frame, risk_score = detect_kidnapping(annotated_frame)

            # 3️⃣ Overlay risk level
            risk_text = "High Risk of Kidnapping!" if risk_score >= 3 else "Safe"
            color = (0, 0, 255) if risk_score >= 3 else (0, 255, 0)
            cv2.putText(annotated_frame, risk_text, (10, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

            # Write frame to output video
            out.write(annotated_frame)

        frame_count += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Kidnapping detection completed. Output saved to: {output_video_path}")


In [8]:
# Change video paths accordingly
video_path = "/content/drive/MyDrive/Source/Kidnap.mp4"  # Replace with test video path
output_video_path = "/content/drive/MyDrive/kidnapping_output1.mp4"

detect_kidnapping_in_video(video_path, output_video_path)


0: 384x640 1 person, 8 cars, 1 truck, 417.6ms
Speed: 33.4ms preprocess, 417.6ms inference, 47.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 truck, 162.1ms
Speed: 4.3ms preprocess, 162.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8 cars, 1 truck, 158.6ms
Speed: 5.1ms preprocess, 158.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8 cars, 1 truck, 163.9ms
Speed: 6.1ms preprocess, 163.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 1 truck, 1 traffic light, 157.7ms
Speed: 4.1ms preprocess, 157.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 1 truck, 1 traffic light, 150.2ms
Speed: 4.9ms preprocess, 150.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 truck, 1 traffic light, 178.5ms
Speed: 3.1ms preprocess, 178.5